In [1]:
import pymssql
import pandas as pd
import psycopg2
import datetime
import psycopg2.extras as extras
import numpy as np

In [2]:
prod_url = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
prod_host = '10.35.12.194' 
prod_port = '5432'
prod_dbname = 'gpadmin'
user_prod="gpspark"
# pwd_prod="gpcustcoe@123"
pwd_prod="spark@456"


# Define the GPDB-Python connection options for PROD 

conn_prod = psycopg2.connect(host=prod_host, port=prod_port, user=user_prod, password=pwd_prod, dbname=prod_dbname)

In [ ]:
list_mot=['num_reference_no','dc_unified_id','customer_id','policy_no','certificate_no','pol_incept_date','pol_exp_date','record_type_desc','product_name','product_cd','renl_cert_no','binddate','plan_type','policy_type','channel','birth_date','city','state','pin_code','times_renewed_cnt','minor_line','sum_insured','premiumamount','total_premium','date_of_registration','all_product_list','age', 'rn']

In [ ]:
list_lta=['num_reference_no','dc_unified_id','customer_id','policy_no','certificate_no','pol_incept_date','pol_exp_date','record_type_desc','product_name','product_cd','renl_cert_no','binddate','plan_type','policy_type','channel','birth_date','city','state','pin_code','sum_insured','times_renewed_cnt','minor_line','premiumamount','total_premium','no_of_lives','risk','lta_dob','id_type','id_value','relationship','age','rn','all_product_list']

In [3]:
list_hlt=['num_reference_no','dc_unified_id','customer_id','policy_no','certificate_no','pol_incept_date','pol_exp_date','record_type_desc','product_name','product_cd','renl_cert_no','binddate','plan_type','policy_type','channel','birth_date','city','state','pin_code','sum_insured','times_renewed_cnt','minor_line','premiumamount','total_premium','member_id','member_entry_date','insured_name','insured_dob','risk','all_product_list']

In [4]:
cursor = conn_prod.cursor()
req="select * from  customermart.temp_ebiz_hmt_final_hlt"
cursor.execute(req)
data1= cursor.fetchall()
df=pd.DataFrame(data1,columns=list_hlt)

In [12]:
df.shape

(220092, 30)

In [24]:
df2=df.copy()

In [25]:
df2.shape

(220092, 30)

In [26]:
df2 = df2.astype({"all_product_list": str, "product_name": str, "num_reference_no": str, "premiumamount":'float', "total_premium":'float', "sum_insured":'float'})

In [27]:
#df2 = df2.astype({"all_product_list": str, "product_name": str, "num_reference_no": str, "relationship": str, "premiumamount":'float', "total_premium":'float', "sum_insured":'float'})

In [ ]:
df2.relationship.fillna(value=np.nan, inplace=True)
df2.id_type.fillna(value=np.nan, inplace=True)
df2.id_value.fillna(value=np.nan, inplace=True)

In [ ]:
#df2 = df2.astype({"all_product_list": str, "product_name": str, "application_no": str, "proposal_no": str, "mobile": str, "premiumamount":'float', "total_premium":'float', "sum_insured":'float'})

In [28]:
df2.shape

(220092, 30)

In [18]:
#df2=df2[df2.premiumamount>1]

In [19]:
df2.shape

(219880, 30)

In [ ]:
#df2.dtypes

In [ ]:
#df2['other_product_name'] = df2['all_product_list'].str.split(',').apply(lambda x:','.join([i for i in x if i not in df2['product_name'].values]))


In [29]:
# making copy of team column
new = df2["channel"].copy()
 
# concatenating team with name column
# overwriting name column
df2["pnc"]= df2["product_name"].str.cat(new, sep ="|")

In [30]:
df2.shape

(220092, 31)

In [31]:
f=lambda x: ', '.join(set(x['all_product_list'].split(',')).difference(set(x['pnc'].split(','))))
df2['other_product_name'] = df2.apply(f, axis=1)

In [32]:
df2.head()

,num_reference_no,dc_unified_id,customer_id,policy_no,certificate_no,pol_incept_date,pol_exp_date,record_type_desc,product_name,product_cd,...,premiumamount,total_premium,member_id,member_entry_date,insured_name,insured_dob,risk,all_product_list,pnc,other_product_name
0,201709290005484,23360411.0,6009618769,0235298353,000000,2017-12-27,2018-12-26,RENEWAL BUSINESS,ACCIDENTSICKNESSHOSPITALCASH,2874,...,4156.00,4904.08,0000009851201040,27/12/2016,SHYAM SUNDER . .,25/11/1976,Spouse,ACCIDENTSICKNESSHOSPITALCASH|DIRECT MARKETING,ACCIDENTSICKNESSHOSPITALCASH|DIRECT MARKETING,
1,202312180077270,133089925.0,6127048308,7000114982,000000,2024-01-18,2025-01-17,RENEWAL BUSINESS,MEDICARE,2888,...,18750.96,22126.14,IDV00032207202034,18/01/2023,Priyanka Shyam Jaiswal,29/05/1988,Self,MEDICARE|E-BUSINESS,MEDICARE|E-BUSINESS,
2,202402220344980,195231095.0,6157587330,7000214423,000000,2024-02-17,2025-02-16,NEW BUSINESS,MEDICARE,2888,...,28696.68,33862.08,IDV00121576801043,17/02/2024,NARESH KUMAR P,25/05/1980,Spouse,MEDICARE|E-BUSINESS,MEDICARE|E-BUSINESS,
3,202401060156807,185841462.0,6153995604,7090000786,000000,2024-01-05,2025-01-04,NEW BUSINESS,TATAAIG HEALTH SUPER CHARGE,2846,...,22406.80,26440.02,IDV00213051602042,05/01/2024,MATTAM GEETHAJALI,03/12/1981,Spouse,TATAAIG HEALTH SUPER CHARGE|E-BUSINESS,TATAAIG HEALTH SUPER CHARGE|E-BUSINESS,
4,202010090176709,22713893.0,6015312079,0235776817,000000,2021-01-30,2022-01-29,RENEWAL BUSINESS,ACCIDENTSICKNESSHOSPITALCASH,2874,...,2096.00,2473.28,ADER0121705449603010,02/01/2018,NEKKALAPUDI JIGNASH,04/06/2007,DependentChild2,ACCIDENTSICKNESSHOSPITALCASH|DIRECT MARKETING,ACCIDENTSICKNESSHOSPITALCASH|DIRECT MARKETING,


In [ ]:
#df2[df2.total_product_count>1]

In [ ]:
#df2.mobile.replace(np.nan,'',regex = True, inplace=True)

In [33]:
df2=df2[['num_reference_no','dc_unified_id','customer_id','policy_no','certificate_no','pol_incept_date','pol_exp_date','record_type_desc','product_name','product_cd','renl_cert_no','binddate','plan_type','policy_type','channel','birth_date','city','state','pin_code','sum_insured','times_renewed_cnt','minor_line','premiumamount','total_premium','member_id','member_entry_date','insured_name','insured_dob','risk','all_product_list','other_product_name']]

In [ ]:
df2=df2[['num_reference_no','dc_unified_id','customer_id','policy_no','certificate_no','pol_incept_date','pol_exp_date','record_type_desc','product_name','product_cd','renl_cert_no','binddate','plan_type','channel','birth_date','city','state','pin_code','sum_insured','times_renewed_cnt','minor_line','premiumamount','total_premium','no_of_lives','risk','lta_dob','id_type','id_value','relationship','age', 'other_product_name']]

In [34]:
df2=df2.drop_duplicates()

In [35]:
df2.notna().sum()/len(df2)*100

num_reference_no      100.000000
dc_unified_id          97.606455
customer_id           100.000000
policy_no             100.000000
certificate_no        100.000000
pol_incept_date       100.000000
pol_exp_date          100.000000
record_type_desc      100.000000
product_name          100.000000
product_cd            100.000000
renl_cert_no           99.985006
binddate              100.000000
plan_type              90.696163
policy_type            54.532650
channel               100.000000
birth_date             88.014103
city                   96.039384
state                  96.039384
pin_code               96.200225
sum_insured            99.677862
times_renewed_cnt      91.324537
minor_line            100.000000
premiumamount         100.000000
total_premium         100.000000
member_id              98.922269
member_entry_date      97.053959
insured_name           99.998183
insured_dob           100.000000
risk                  100.000000
all_product_list      100.000000
other_prod

In [36]:
df2.shape

(220092, 31)

In [37]:
df2.to_csv('Ebiz_data_hlt_new.csv')